In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sns
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from network import create_model, create_dataloaders

In [12]:
# 检查PyTorch是否正确编译了CUDA
print(f"PyTorch是否编译了CUDA: {torch.cuda.is_available()}")
print(f"PyTorch CUDA版本: {torch.version.cuda}")

PyTorch是否编译了CUDA: True
PyTorch CUDA版本: 12.6


In [ ]:
# 1. 选择训练方式
sizes = [128, 256, 512, 1024, 2048, 4096] # 基于符号长度进行训练
snr_list = [-10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50] # 基于信噪比进行训练
items = snr_list

In [ ]:
# 加载数据
X_train, y_train = {}, {}
X_test, y_test = {}, {}

for item in items:
# for items in sizes:
    # 训练集
    df_train = shuffle(pd.read_csv(f'../preprocessed/combined_features_{item}.csv'), random_state=42)
    y_train = df_train['label']
    X_train = df_train.drop('label', axis=1).applymap(lambda x: complex(x)).values
    
    # 测试集
    df_test = shuffle(pd.read_csv(f'../preprocessed/combined_features_{item}_test.csv'), random_state=42)
    y_test = df_test['label']
    X_test = df_test.drop('label', axis=1).applymap(lambda x: complex(x)).values

C:\Users\admin\AppData\Local\Temp\ipykernel_27212\37876306.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_train = df_train.drop('label', axis=1).applymap(lambda x: complex(x)).values
C:\Users\admin\AppData\Local\Temp\ipykernel_27212\37876306.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_test = df_test.drop('label', axis=1).applymap(lambda x: complex(x)).values
C:\Users\admin\AppData\Local\Temp\ipykernel_27212\37876306.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_train = df_train.drop('label', axis=1).applymap(lambda x: complex(x)).values
C:\Users\admin\AppData\Local\Temp\ipykernel_27212\37876306.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_test = df_test.drop('label', axis=1).applymap(lambda x: complex(x)).values
C:\Users\admin\AppData\Local\Temp\ipykernel_27212\37876306.py:10: FutureWarning: DataFra

In [15]:
# 2. 创建数据加载器
batch_size = 32
train_loader, test_loader, (scaler_real, scaler_imag) = create_dataloaders(X_train, y_train, X_test, y_test, batch_size)

In [16]:
# 3. 创建模型
num_features = X_train.shape[1]
num_classes = len(np.unique(y_train))
model = create_model(num_features, num_classes)

In [17]:
# 4. 设置训练参数
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 确保模型在GPU上
model = create_model(num_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)

使用设备: cuda


In [ ]:
# 5. 训练模型
num_epochs = 30
best_acc = 0.0

accuracies = []


for item in items:
    print(f"\n=== Training for items {item} ===")
    for epoch in range(1, num_epochs + 1):
        # 训练阶段
        model.train()
        total_loss = 0
        
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # 验证阶段
        model.eval()
        correct = 0
        total = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                outputs = model(x)
                _, predicted = torch.max(outputs.data, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(y.cpu().numpy())
        
        acc = correct / total
        scheduler.step(acc)
        
        if acc > best_acc:
            best_acc = acc
            # 保存最佳模型
            # torch.save(model.state_dict(), f'./model_save/ComplexCNN_{item}.pth')
        
        print(f'Epoch {epoch}/{num_epochs}: Loss={total_loss/len(train_loader):.4f}, Acc={acc:.4f}, Best Acc={best_acc:.4f}')

    print(f"最终最佳准确率: {best_acc:.4f}")


=== Training for items -10 ===
Epoch 1/30: Loss=0.7435, Acc=0.5460, Best Acc=0.5460
Epoch 2/30: Loss=0.6934, Acc=0.5393, Best Acc=0.5460
Epoch 3/30: Loss=0.6932, Acc=0.5600, Best Acc=0.5600
Epoch 4/30: Loss=0.6848, Acc=0.5767, Best Acc=0.5767
Epoch 5/30: Loss=0.6860, Acc=0.6045, Best Acc=0.6045
Epoch 6/30: Loss=0.6876, Acc=0.5765, Best Acc=0.6045
Epoch 7/30: Loss=0.6806, Acc=0.5890, Best Acc=0.6045
Epoch 8/30: Loss=0.6798, Acc=0.5500, Best Acc=0.6045
Epoch 9/30: Loss=0.6766, Acc=0.5617, Best Acc=0.6045
Epoch 10/30: Loss=0.6671, Acc=0.6130, Best Acc=0.6130
Epoch 11/30: Loss=0.6637, Acc=0.6058, Best Acc=0.6130
Epoch 12/30: Loss=0.6633, Acc=0.6135, Best Acc=0.6135
Epoch 13/30: Loss=0.6592, Acc=0.6002, Best Acc=0.6135
Epoch 14/30: Loss=0.6580, Acc=0.6362, Best Acc=0.6362
Epoch 15/30: Loss=0.6587, Acc=0.6028, Best Acc=0.6362
Epoch 16/30: Loss=0.6577, Acc=0.6225, Best Acc=0.6362
Epoch 17/30: Loss=0.6558, Acc=0.6292, Best Acc=0.6362
Epoch 18/30: Loss=0.6532, Acc=0.6365, Best Acc=0.6365
Epoch

In [ ]:
# 6. 绘制混淆矩阵（可选）
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('混淆矩阵', fontproperties="SimSun")
plt.ylabel('真实类别', fontproperties="SimSun")
plt.xlabel('预测类别', fontproperties="SimSun")
plt.xticks(range(4), ['bladerf', 'hackrf0', 'hackrf1', 'limesdr'])
plt.yticks(range(4), ['bladerf', 'hackrf0', 'hackrf1', 'limesdr'])
plt.savefig(f'./img/confusion_matrix_{item}.png', dpi=300, bbox_inches='tight')
plt.show()